In [1]:
import pandas as pd
import numpy as np
import re
import os
from collections import Counter

In [2]:
directory = r'C:/Users/gdoku/YandexDisk/Компания/'\
    +'06 Договоры (прочие)/Договор оказания услуг ООО Мани Френдс/'\
    +'Выписки'
files = pd.Series(os.listdir(directory))

statment_files = list(files[files.str.contains('Выписка')])
schedule_files = list(files[files.str.contains('Оферта')])

In [3]:
counter = 0
for file in statment_files:
    text = open(os.path.join(directory, file), 'r', encoding='utf-8').read()
    text = text.replace('\n', ';').replace(
        ';Успешно;', '\n').replace(';Успешно', '').split('\n')
    if counter == 0:
        statment = pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()
        counter += 1
    else:
        statment = pd.concat([statment, pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()]).reset_index(
                drop=True).drop_duplicates()

In [4]:
statment.raw = statment.raw.str.split(';')
statment['date'] = pd.to_datetime(statment.raw.apply(lambda x: x[0]), format='%d.%m.%Y')
statment['time'] = pd.to_datetime(statment.raw.apply(lambda x: x[0]+' '+x[1]), format='%d.%m.%Y %H:%M:%S')
statment['type'] = statment.raw.apply(lambda x: ' '.join(x[2].split(' ')[0:4]))
statment['amount'] = statment.raw.apply(lambda x: float(x[3][:-2].replace(' ', '')))
statment['description'] = statment.raw.apply(lambda x: x[4] if len(x)==5 else x[2])
statment = statment.sort_values(by='time').drop(columns=['raw','time']).reset_index(drop=True)

In [5]:
types = ['Возврат займа', 'Возврат процентов по займу',
                         'Вывод средств', 'Пополнение средств', 'Перевод средств из резерва']
statment = statment[statment.type.isin(types)].reset_index(drop=True)
statment['debit'] = None
statment['credit'] = None
statment['loan'] = None
statment['borrower'] = None
statment.loc[statment.type.str.contains('Вывод средств'), 'debit'] = 51002
statment.loc[statment.type.str.contains('Вывод средств'), 'credit'] = 55001
statment.loc[statment.type.str.contains('Пополнение средств'), 'debit'] = 55001
statment.loc[statment.type.str.contains('Пополнение средств'), 'credit'] = 51002
statment.loc[statment.type.str.contains('Возврат займа'), 'debit'] = 55001
statment.loc[statment.type.str.contains('Возврат займа'), 'credit'] = None
statment.loc[statment.type.str.contains('Возврат процентов по займу'), 'debit'] = 55001
statment.loc[statment.type.str.contains('Возврат процентов по займу'), 'credit'] = None
statment.loc[statment.type.str.contains('Перевод средств из резерва'), 'debit'] = None
statment.loc[statment.type.str.contains('Перевод средств из резерва'), 'credit'] = 55001

In [6]:
pattern_loan = re.compile('№\d{3,4}')
pattern_borrower = re.compile('Заемщик .+\. С')

statment.loan = None
statment.loc[~statment.type.isin(['Пополнение средств', 'Вывод средств']), 'loan'] = statment.description.apply(
    lambda x: str(re.findall(pattern_loan, x))[3:-2])

statment.borrower = None
statment.loc[~statment.type.isin(['Пополнение средств', 'Вывод средств',
                                  'Перевод средств из резерва']), 'borrower'] = statment.description.apply(
    lambda x: str(re.findall('«.+»',str(re.findall(pattern_borrower, x))[10:-5]))[3:-3])

In [7]:
statment

,date,type,amount,description,debit,credit,loan,borrower
0,2020-01-22,Пополнение средств,56000.00,Внесение средств на баланс по счету №ENV-44736...,55001,51002,None,None
1,2020-02-04,Перевод средств из резерва,56000.00,Перевод средств из резерва по оферте №487,None,55001,487,None
2,2020-02-25,Пополнение средств,170000.00,Внесение средств на баланс по счету №ENV-93980...,55001,51002,None,None
3,2020-02-27,Перевод средств из резерва,170000.00,Перевод средств из резерва по оферте №569,None,55001,569,None
4,2020-03-04,Возврат займа,8933.26,Возврат основного долга по договору займа №487...,55001,None,487,КАЛИТА
5,2020-03-04,Возврат процентов по займу,980.00,Получение процентного дохода по договору займа...,55001,None,487,КАЛИТА
6,2020-03-04,Вывод средств,9913.26,Запрос на вывод средств с баланса на Расчетный...,51002,55001,None,None
7,2020-03-27,Возврат займа,12914.54,Возврат основного долга по договору займа №569...,55001,None,569,КАРОЛИНА
8,2020-03-27,Возврат процентов по займу,2833.34,Получение процентного дохода по договору займа...,55001,None,569,КАРОЛИНА
9,2020-03-27,Вывод средств,15747.88,Запрос на вывод средств с баланса на Расчетный...,51002,55001,None,None


In [14]:
counter = 0
for file in schedule_files:
    text = open(os.path.join(directory, file), 'r', encoding='utf-8').read()
    text = text.replace('\n', ';').replace(
        ';Успешно;', '\n').replace(';Успешно', '').split('\n')
    if counter == 0:
        schedule = text
#         pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()
        counter += 1
    else:
        break
#         schedule = pd.concat([schedule, pd.DataFrame(text).rename(columns={0:'raw'}).drop_duplicates()]).reset_index(
#                 drop=True).drop_duplicates()

In [29]:
pd.DataFrame(np.array(schedule[0].split(';')[11:-4]).reshape(-1,4))

,0,1,2,3
0,04.03.2020,9 913.26 ₽,980.00 ₽,8 933.26 ₽
1,04.04.2020,9 913.27 ₽,823.67 ₽,9 089.60 ₽
2,04.05.2020,9 913.26 ₽,664.60 ₽,9 248.66 ₽
3,04.06.2020,9 913.27 ₽,502.75 ₽,9 410.52 ₽
4,04.07.2020,9 913.26 ₽,338.06 ₽,9 575.20 ₽
5,04.08.2020,9 913.27 ₽,170.50 ₽,9 742.77 ₽
